In [3]:
# load data
import pandas as pd

df = pd.read_csv("../data/classpass_data.csv")

In [4]:
# feature engineering

X = pd.get_dummies(df.drop(columns = ["booked", "prob"]), drop_first = True)

y = df["booked"]

X = pd.get_dummies(df[["credit_price", "current_streak", "social_score", "days_since_last_booking", "persona", "class_type"]], drop_first = True)

y = df["booked"]

X = pd.get_dummies(df[["credit_price", "current_streak", "social_score", "class_type"]], drop_first = True)

In [5]:
# train test split

from sklearn.model_selection import train_test_split

X_traian, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [6]:
# train model

from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter = 1000)
model.fit(X_train, y_train)

NameError: name 'X_train' is not defined

In [ ]:
# evaluate

from sklearn.metrics import acuracy_score, roc_auc_score

y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)
roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])

In [ ]:
# feature importance

pd.Series(model.coef_[0], index = X.columns).sort_values()